# Step 1: Data Preparation - Data Sourcing and Cleaning

In [2]:
# Data sourcing and cleaning - missing values, duplicate values, formatting, headers ,anomalies/outliers
import pandas as pd
df = pd.read_csv('../data/raw_data/Children-and-young-ppl-asthma-organisational-audit-2019-20-Data.csv')
print("Original Data")
print(df.head(10))
print("\n \n")
print(df.tail(10))
#Returning how many rows in the df
print("\n The amount of organisations in this data set is",df.count().values[0],".") 


Original Data
  orgcode                          description  \
0     ADD               Addenbrooke's Hospital   
1     AEI        Royal Albert Edward Infirmary   
2     AIR            Airedale General Hospital   
3     ALC  Royal Alexandra Children's Hospital   
4     BAR   Barnsley District General Hospital   
5     BAS                    Basildon Hospital   
6     BAT           Royal United Hospital Bath   
7     BCH       Birmingham Children's Hospital   
8     BED                     Bedford Hospital   
9     BFH                Broomfield Chelmsford   

                                               trust  country emergency_adm  \
0  Cambridge University Hospitals NHS Foundation ...  England             -   
1  Wrightington, Wigan and Leigh NHS Foundation T...  England           400   
2                      Airedale NHS Foundation Trust  England           110   
3  Brighton and Sussex University Hospitals NHS T...  England            10   
4             Barnsley Hospital NHS Foun

Assessing the quality - high quality data set and how does it reflect real life data. It is reliable as it is an open data set which is provided by a creditable source. This is an external data source provided by the data.go.uk of the information of asthma in children and young adults from 2018 - 2019. It is a structured data as it carried out the following characteristics : well organised, table form, and the tables can be related in a database schema. 

From my inital inspection I have noticed that there are a few rows which have empty values. Majority of the rows which have empty values mostly have value which are empty in the first couple of rows (A-G) which is an interger of the type of admission. I believe that it would be benefical to drop these values because the admission data is one of the important columns in the data set, and as these reigons of hospitals differ i would not think i would be benefical to make adverages of these admissions. 

Via inspection I have noticed that some values which are - are also equlient to Nan values, but since python is dropna() function only detects nan values I would have to convert the NaN or Null values 

In [4]:
# Data Cleaning - missing values
# Checking if there are missing values 
for row in df.iterrows():
    print(df.isna())
# goes through each row in the 
# for row in df.row:
    #df[row].dropna(inplace=True)
    #print(df.count().values[0])

     orgcode  description  trust  country  emergency_adm  respiratory_adm  \
0      False        False  False    False          False            False   
1      False        False  False    False          False            False   
2      False        False  False    False          False            False   
3      False        False  False    False          False            False   
4      False        False  False    False          False            False   
..       ...          ...    ...      ...            ...              ...   
137    False        False  False    False          False            False   
138    False        False  False    False          False            False   
139    False        False  False    False          False            False   
140    False        False  False    False          False            False   
141    False        False  False    False          False            False   

     asthma_adm  admsperbed  respadmsperbed  asthmaadmsper1000adms  ...  \


The statement which I have chosen takes into consideration the amount of na values which is in each row 

In [ ]:
# Data transformation - normalise, Discretize or aggregate , new attributes 


In [ ]:
# Data reduction